In [65]:
import scipy as sp
import scipy.stats as sps
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import seaborn as sns
import jenkspy as jp
import math
from sklearn.manifold import MDS
from itertools import pairwise, combinations,product

In [66]:
df = pd.read_csv('./results-survey673776.csv')
pd.set_option('display.max_rows', 500)

In [83]:
def precision(answer, targets):
    num_tp = len(set(answer).intersection(set(targets)))
    return num_tp / len(answer)

precision([1,2,3], [2,1])

0.6666666666666666

In [70]:
df = df[pd.notna(df['submitdate'])]
# TODO exclude more based on vision, dyslexia, age, control questions...

conditions = list(product(['D1','D2','D3', 'D4'], ['Ph','Po'], ['W1','W5','W0']))
experiment_questions = list(map(lambda x:  f"exp{''.join(x)}",conditions))
experiment_questions

['expD1PhW1',
 'expD1PhW5',
 'expD1PhW0',
 'expD1PoW1',
 'expD1PoW5',
 'expD1PoW0',
 'expD2PhW1',
 'expD2PhW5',
 'expD2PhW0',
 'expD2PoW1',
 'expD2PoW5',
 'expD2PoW0',
 'expD3PhW1',
 'expD3PhW5',
 'expD3PhW0',
 'expD3PoW1',
 'expD3PoW5',
 'expD3PoW0',
 'expD4PhW1',
 'expD4PhW5',
 'expD4PhW0',
 'expD4PoW1',
 'expD4PoW5',
 'expD4PoW0']

In [86]:
exp = df[experiment_questions]

d1 = pd.read_csv('./dataset/ds_dataset1.csv')
d2 = pd.read_csv('./dataset/ds_dataset2.csv')
d3 = pd.read_csv('./dataset/ds_dataset3.csv')
d4 = pd.read_csv('./dataset/ds_dataset4.csv')

target_d1 = [i for i in d1[d1['cluster']==1].index.tolist() if i != 40 ]
target_d2 = d2[d2['S4']==1].index.tolist()
target_d3 = d3[(d3['S0']==1) & (d3['S2']==1)].index.tolist()
target_d4 = [ i for i in d4[(d4['cluster']==1) & ((d4['S1']==1)|(d4['S2']==1))].index.tolist() if i != 37]

def get_target(q):
    match q[4]:
        case '1': return target_d1
        case '2': return target_d2
        case '3': return target_d3
        case '4': return target_d4
    raise BaseException(q[4])


for q in experiment_questions:
    exp[f"{q}_t"] = pd.NA
    exp[f"{q}_a"] = pd.NA
    for i in exp.index:
        timed_response = exp.loc[i,q]
        exp.loc[i,f"{q}_t"] = int(timed_response.split(';')[1])
        response = [int(i) for i in timed_response.split(';')[0].split(',')]
        exp.loc[i,f"{q}_a"] = precision(response, get_target(q))

exp = exp.drop(columns=experiment_questions)
exp

exp_a = exp[list(map(lambda q: f"{q}_a",experiment_questions))]
exp_t = exp[list(map(lambda q: f"{q}_t",experiment_questions))]

exp_t.to_csv('./exp_t.csv')

/var/folders/d9/8l4xsjcd5_59v2f35x1y8dgm0000gn/T/ipykernel_37222/2671524494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp[f"{q}_t"] = pd.NA
/var/folders/d9/8l4xsjcd5_59v2f35x1y8dgm0000gn/T/ipykernel_37222/2671524494.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp[f"{q}_a"] = pd.NA
/var/folders/d9/8l4xsjcd5_59v2f35x1y8dgm0000gn/T/ipykernel_37222/2671524494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde